## Smart Signatures with Transaction Groups
#### 06.3.5 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-22

* Smart Signatures with more than 1 transaction
* Combine conditions across transactions

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the 5 accounts and the Purestake credentials

In [72]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [73]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import pandas as pd

In [74]:
from pyteal import *

In [75]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [76]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


## The Split Payment
* Classical Smart Contract
* Example
    * Two business partners agree to split all revenues in a fixed percentage
    * The smart contract is the business acount, into which customers have to pay
    * Both business partners can initiate a payout, but only in the fixed percentage
* Other examples
    * Fixed tax rate
    * Sales commission

#### Step 0: Get the status before the transaction
* Also fund accounts if need be
* https://bank.testnet.algorand.network
* https://testnet.algoexplorer.io/dispenser

In [77]:
# Get the holdings of MyAlgo and Bob separately
alice_holding=asset_holdings_df(algod_client, Alice['public'])
bob_holding=asset_holdings_df(algod_client, Bob['public'])
# Merge in one data.frame using pandas merge
pd.merge(alice_holding, bob_holding,  how="outer", on=["asset-id", "unit", "name", "decimals"], suffixes=['Alice','Bob'])

,amountAlice,unit,asset-id,name,decimals,amountBob
0,45.378991,ALGO,0,Algorand,6,64.286088
1,200.000000,USDC,10458941,USDC,6,100.000000
2,0.100000,WSC,66504861,WSC coin,2,NaN
3,0.000000,WSC,66505040,WSC coin,2,0.770000
4,100.000000,ALICE,66712019,Alice's Tempcoin,1,NaN
5,100.000000,ALICE,66712340,Alice's Tempcoin,1,NaN
6,1.000000,ALICEART,69394953,Alice's First Portrait 001,0,NaN
7,0.000000,VtC,70161280,VoteCoin,2,2.000000
8,0.000000,VtC,70166124,VoteCoin,2,3.000000
9,NaN,WSC,66709453,Peters WSC coin,2,10.000000


#### Step 1a: Write down the conditions as a PyTeal program
* Alice and Bob are business partners
* Alice gets 3/4 of the proceeds
* Bob gets 1/4 of the proceeds

In [87]:
split_cond = And( 
    Gtxn[0].sender() == Gtxn[1].sender(),                        # Both payments come from the same address
    Gtxn[0].receiver() == Addr(Alice['public']),                 # Payment 0 to Alice
    Gtxn[1].receiver() == Addr(Bob['public']),                   # Payment 1 to Bob
    Gtxn[0].amount() == Int(4) * (Gtxn[0].amount() + Gtxn[1].amount()) / Int(5)    # Alice_amount = 3/4 * Total_amount
    )

fee_cond = And( 
    Gtxn[0].fee() <= Int(1000),                                    # No fee attack
    Gtxn[1].fee() <= Int(1000)                                     # No fee attack
    )

safety_cond = And( 
    Global.group_size() == Int(2),                                  # Exactly 2 transactions
    Gtxn[0].type_enum() == TxnType.Payment,                         # Both are PaymentTxn
    Gtxn[1].type_enum() == TxnType.Payment,
    Gtxn[0].rekey_to() == Global.zero_address(),                    # No rekey attach
    Gtxn[1].rekey_to() == Global.zero_address(),
    Gtxn[0].close_remainder_to() == Global.zero_address(),          # No close_to attack
    Gtxn[1].close_remainder_to() == Global.zero_address()
    )
    
split_pyteal = And(
    split_cond, 
    fee_cond, 
    safety_cond
    )

#### Step 1b: Pyteal -> Teal

In [88]:
split_teal = compileTeal(split_pyteal, Mode.Signature, version=3)
print(split_teal)

#pragma version 3
gtxn 0 Sender
gtxn 1 Sender
==
gtxn 0 Receiver
addr HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
==
&&
gtxn 1 Receiver
addr O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
==
&&
gtxn 0 Amount
int 4
gtxn 0 Amount
gtxn 1 Amount
+
*
int 5
/
==
&&
gtxn 0 Fee
int 1000
<=
gtxn 1 Fee
int 1000
<=
&&
&&
global GroupSize
int 2
==
gtxn 0 TypeEnum
int pay
==
&&
gtxn 1 TypeEnum
int pay
==
&&
gtxn 0 RekeyTo
global ZeroAddress
==
&&
gtxn 1 RekeyTo
global ZeroAddress
==
&&
gtxn 0 CloseRemainderTo
global ZeroAddress
==
&&
gtxn 1 CloseRemainderTo
global ZeroAddress
==
&&
&&


#### Step 1c: Teal -> Bytecode for AVM

In [89]:
Split = algod_client.compile(split_teal)
Split

{'hash': 'BLCGVWDB3RS3DCFFCEN65MKOQDD4WW3LKF57OKIGGRDMSGRHPCVZ5Q72BI',
 'result': 'AyAFBAXoBwIBJgIgOibwATw6gNY/xKYBK46TEv3dz/BjTpX1rhr9chgs4oIgdqS4vVxHJWo4WDGUc9BFSkvXROVf/YdzZcTD89HfAcYzAAAzAQASMwAHKBIQMwEHKRIQMwAIIjMACDMBCAgLIwoSEDMAASQOMwEBJA4QEDIEJRIzABAhBBIQMwEQIQQSEDMAIDIDEhAzASAyAxIQMwAJMgMSEDMBCTIDEhAQ'}

### The split payment is now ready
* We only need to communicate the hash to customers

#### Step 2: A customer makes a payment
* Dina buys something from the Alice_Bob_Company
* She pays 5 Algos into the company account

In [90]:
# Step 2.1: prepare transaction
sp = algod_client.suggested_params()
amt = int(5*1e6)
txn = transaction.PaymentTxn(sender=Dina['public'], sp=sp, 
                             receiver=Split['hash'], amt=amt)

# Step 2.(2+3+4): sign and send and wait ...
stxn = txn.sign(Dina['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19824838.
Waiting for round 19824838 to finish.
Transaction 3UVPFX5DBW47JXEUJFYTQONQLOAA5KT4KTOKR3TOVCOI3ZHYKZXA confirmed in round 19824839.


#### Step 3: Payout request
* Alice or Bob or anybody actually can make a payout request
* The only thing that matter is that 3/4 go to Alice and 1/4 goes to Bob
* Consider the TX fees and min holdings in the contract

In [93]:
### Step 3.1: prepare and create TX group
sp = algod_client.suggested_params()

total_amt = 4.8       # total withdrawl
amt_1 = int(4/5 * 4.8 * 1E6)  # Alice share in microalgos
amt_2 = int(1/5 * 4.8 * 1E6)  # Bob share in microalgos

txn_1 = PaymentTxn(sender=Split['hash'],sp=sp,receiver=Alice['public'],amt=amt_1)
txn_2 = PaymentTxn(sender=Split['hash'],sp=sp,receiver=Bob['public'],amt=amt_2)

gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid
txn_2.group = gid

# Step 3.2a ask Smart Signature to sign txn_1
encodedProg = Split['result'].encode()              
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_1 = LogicSigTransaction(txn_1, lsig)

# Step 3.2b ask Smart Signature to sign txn_2
encodedProg = Split['result'].encode()              
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

# Step 3.3: assemble transaction group and send
signed_group =  [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

# Step 3.4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  19824852.
Waiting for round 19824852 to finish.
Waiting for round 19824853 to finish.
Transaction W7CNBOJ2GHBEY4QXK67VUVRGZH5JMXA7YO3KXP2C7YDHCOPZOWKA confirmed in round 19824854.


## Appendix: how to calcualte the splitting condition

The splitting condition is written in a strange order

```python
    Gtxn[0].amount() == Int(3) * (Gtxn[0].amount()+Gtxn[1].amount()) / Int(4)  

```

This (mathematically identical) will not work

```python
    Gtxn[0].amount() == Int(3)/Int(4) * (Gtxn[0].amount()+Gtxn[1].amount()) 
```

In [84]:
a = 750
b = 250

In [85]:
# Version 1
int(a) == int( int(3 * (int(a) + int(b)) ) / 4)

True

In [86]:
# Version 2
int(a) == int(3/4) * ( int(a) + int(b))

False

#### CONCLUSION: (Py)TEAL uses integer calucaltions for every single step!

## Exercise
* Change the Smart Signature (and the withdrawl transaction) so that Alice gets 80% and Bob gets 20%

## Exercise
* Discuss: How would we deal with fractions like 2/3 and 1/3, that cannot be easily expressed in percentages?